# Import libs

In [1]:
from tokenizers import Tokenizer, pre_tokenizers, trainers, models
from datasets import load_dataset

ds = load_dataset("ncduy/mt-en-vi")

In [144]:
ds

DatasetDict({
    train: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 2884451
    })
    validation: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 11316
    })
    test: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 11225
    })
})

In [2]:
ds.remove_columns(["source"])

DatasetDict({
    train: Dataset({
        features: ['en', 'vi'],
        num_rows: 2884451
    })
    validation: Dataset({
        features: ['en', 'vi'],
        num_rows: 11316
    })
    test: Dataset({
        features: ['en', 'vi'],
        num_rows: 11225
    })
})

# Tokenize / Preprocessing

In [ ]:
# word - based
tokenizer_en = Tokenizer(models.WordLevel(unk_token="<unk>"))
tokenizer_vi = Tokenizer(models.WordLevel(unk_token="<unk>"))
tokenizer_en.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer_vi.pre_tokenizer = pre_tokenizers.Whitespace()
trainer = trainers.WordLevelTrainer(
    vocab_size=50000,
    min_frequency=2,
    special_tokens=["<pad>", "<unk>", "<bos>", "<eos>"],
)
# train tokenizer   
tokenizer_en.train_from_iterator(ds["train"]["en"], trainer)
tokenizer_vi.train_from_iterator(ds["train"]["vi"], trainer)
# tokenizer
tokenizer_en.save("tokenizer_en.json")
tokenizer_vi.save("tokenizer_vi.json")

# Build vocabulary

In [ ]:
from transformers import PreTrainedTokenizerFast

MAX_LEN = 50

# Load tokenizer
tokenizer_en = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_en.json",
    unk_token="<unk>",
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>",
)

tokenizer_vi = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_vi.json",
    unk_token="<unk>",
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>",
)

def preprocess_function(examples):
    src_texts = examples["en"]
    tgt_texts = ["<bos>" + sent + "<eos>" for sent in examples["vi"]]
    src_encodings = tokenizer_en(
        src_texts, padding="max_length", truncation=True, max_length=MAX_LEN
    )
    tgt_encodings = tokenizer_vi(
        tgt_texts, padding="max_length", truncation=True, max_length=MAX_LEN
    )
    return {
        "input_ids": src_encodings["input_ids"],
        "labels": tgt_encodings["input_ids"],
    }


preprocessed_ds = ds.map(preprocess_function, batched=True)

def is_valid_sample(sample):
    return any(token != 0 for token in sample["input_ids"])

preprocessed_ds = preprocessed_ds.filter(is_valid_sample)


In [35]:
preprocessed_ds['train'][20]

{'en': 'Smallpox also ravaged Mexico in the 1520s, killing 150,000 in Tenochtitlán alone, including the emperor, and Peru in the 1530s, aiding the European conquerors.',
 'vi': 'Bệnh đậu mùa cũng tàn phá México vào những năm 1520, chỉ riêng người Tenochtitlán đã có hơn 150.000 người chết, gồm cả quốc vương, và Peru vào những năm 1530, nhờ đó hỗ trợ cho những người châu Âu đi chinh phục.',
 'source': 'WikiMatrix v1',
 'input_ids': [1,
  85,
  18437,
  1270,
  12,
  6,
  1,
  5,
  1520,
  2447,
  5,
  196,
  12,
  1,
  727,
  5,
  253,
  6,
  2390,
  5,
  10,
  3480,
  12,
  6,
  1,
  5,
  17500,
  6,
  675,
  34857,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'labels': [2,
  1918,
  1955,
  530,
  61,
  1336,
  517,
  2668,
  29,
  28,
  20,
  13604,
  5,
  76,
  605,
  17,
  1,
  13,
  10,
  74,
  2341,
  4,
  492,
  17,
  273,
  5,
  255,
  73,
  158,
  905,
  5,
  8,
  2392,
  29,
  28,
  20,
  11487,
  5,
  1115,
  22,
  851,

# Modeling

In [2]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig

# Pretrained Model

## BART

In [ ]:
from transformers import AutoTokenizer

MAX_LEN = 50 

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

def preprocess_function(examples):
    src_texts = examples["en"]
    tgt_texts = examples["vi"]
    src_encodings = tokenizer(src_texts, padding="max_length", truncation=True, max_length=MAX_LEN)
    tgt_encodings = tokenizer(tgt_texts, padding="max_length", truncation=True, max_length=MAX_LEN)
    return {
        "input_ids": src_encodings["input_ids"],
        "labels": tgt_encodings["input_ids"],
    }

preprocessed_ds = ds.map(preprocess_function, batched=True)

## mBART

In [4]:
from transformers import MBart50TokenizerFast

MAX_LEN = 50  

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="vi_VN")

def preprocess_function(examples):
    src_texts = examples["en"]
    tgt_texts = examples["vi"]
    src_encodings = tokenizer(src_texts, padding="max_length", truncation=True, max_length=MAX_LEN)
    with tokenizer.as_target_tokenizer():
        tgt_encodings = tokenizer(tgt_texts, padding="max_length", truncation=True, max_length=MAX_LEN)
    return {
        "input_ids": src_encodings["input_ids"],
        "labels": tgt_encodings["input_ids"],
    }


preprocessed_ds = ds.map(preprocess_function, batched=True)

c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/11225 [00:00<?, ? examples/s]

c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [24]:
tokenizer.convert_ids_to_tokens(250004)

'en_XX'

In [ ]:
preprocessed_ds['train'][20]['input_ids']

In [5]:
from transformers import MBartForConditionalGeneration

model_mbart = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50')

# Testing

In [6]:
input_ids = torch.tensor([preprocessed_ds["train"][10]["input_ids"]])
labels = torch.tensor([preprocessed_ds["train"][10]["labels"]])
pred = model_mbart(input_ids=input_ids, labels=labels)


In [ ]:
pred

# Trainer

In [8]:
# Disable wandb
import os

os.environ["WANDB_DISABLED"] = "true"
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Training
training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart-50-en-vi",
    logging_dir="logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    learning_rate=2e-05,
    save_total_limit=1,
    predict_with_generate=True,
    fp16=True,
    lr_scheduler_type="cosine",
    weight_decay=0.05,
    gradient_checkpointing=True,
    gradient_accumulation_steps=2,
    #report_to="wandb",
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [9]:
trainer = Seq2SeqTrainer(
    model=model_mbart,
    args=training_args,
    train_dataset=preprocessed_ds["train"],
    eval_dataset=preprocessed_ds["validation"],
    tokenizer=tokenizer,
)
trainer.train()

c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/90139 [00:00<?, ?it/s]

c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strateg

{'loss': 0.6184, 'grad_norm': 1.205190658569336, 'learning_rate': 6.219338429813704e-12, 'epoch': 1.0}


  0%|          | 0/708 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


{'eval_loss': 0.4920928478240967, 'eval_runtime': 27.8232, 'eval_samples_per_second': 406.711, 'eval_steps_per_second': 25.446, 'epoch': 1.0}
{'train_runtime': 39861.936, 'train_samples_per_second': 72.361, 'train_steps_per_second': 2.261, 'train_loss': 0.6184480098098493, 'epoch': 1.0}


TrainOutput(global_step=90139, training_loss=0.6184480098098493, metrics={'train_runtime': 39861.936, 'train_samples_per_second': 72.361, 'train_steps_per_second': 2.261, 'total_flos': 3.052232971124736e+17, 'train_loss': 0.6184480098098493, 'epoch': 0.999994453042229})

In [43]:
input=tokenizer("I am going to bed", return_tensors="pt")['input_ids'].to(model_mbart.device)

In [44]:
output=model_mbart.generate(input, max_length=50, num_beams=5, early_stopping=True, temperature=1.0, do_sample=True, forced_bos_token_id=tokenizer.lang_code_to_id["vi_VN"])

In [51]:
tokenizer.convert_ids_to_tokens(4724)

'▁đang'

In [46]:
output

tensor([[     2, 250024,    384,  38068,   4724,   2467,  27421,   4600,      5,
              2]], device='cuda:0')

In [49]:
tokenizer.decode(output[0], skip_special_tokens=True)

'Tớ đang đi ngủ đây.'

In [20]:
trainer.push_to_hub(commit_message="mBART-50 EN-VI") 

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/binhphap5/mbart-50-en-vi/commit/41bc2dee30026ba8283440d56276a8f174ce3b0c', commit_message='mBART-50 EN-VI', commit_description='', oid='41bc2dee30026ba8283440d56276a8f174ce3b0c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/binhphap5/mbart-50-en-vi', endpoint='https://huggingface.co', repo_type='model', repo_id='binhphap5/mbart-50-en-vi'), pr_revision=None, pr_num=None)

In [5]:
torch.cuda.empty_cache()

In [22]:
trainer.evaluate(preprocessed_ds["test"])

  0%|          | 0/702 [00:00<?, ?it/s]

{'eval_loss': 0.49249181151390076,
 'eval_runtime': 28.4953,
 'eval_samples_per_second': 393.924,
 'eval_steps_per_second': 24.636,
 'epoch': 0.999994453042229}

In [11]:
from transformers import pipeline
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
translator = pipeline(
    model="binhphap5/mbart-50-en-vi",
    device=device,
)
translator(
    "A cat is going to the moon with the astronauts",
    max_length=50,
    num_beams=5,
    early_stopping=True,
    temperature=1,
    do_sample=True,
)[0]["generated_text"]

'Một con mèo sẽ lên mặt trăng với các phi hành gia'